In [ ]:
!pip install wandb -qqq
!pip install tensorflow-addons

In [1]:
class Config:
    N_SPLITS = 10
    N_EPOCHS = 400
    BATCH_SIZE = 128
    CAT_DIM = 256
    START_LR = 2e-4
    END_LR = 1e-6
    DENSE_SIZE = 512
    SEQUENCE_LENGTH = 39
    DROPOUT = 0.75
    EMBEDDING_DROPOUT = 0.7
    WANDB_ENABLED = False
    WANDB_API_KEY = '4dad92fd17d935955886d8b751df8aa51f7a8523'
    VER = 'v55'
    SAMPLE_TRAIN_BREATH_IDS = None
    SAMPLE_TEST_BREATH_IDS = None
    DATA_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'
    OUTPUT_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'
    START_FOLD = 0
    END_FOLD = 1
    RANDOM_STATE=523

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, StandardScaler
from tqdm import tqdm
from time import time
import os
import gc

# import tensorflow as tf
from functools import partial

# import tensorflow.keras.backend as K
# from tensorflow.keras.models import load_model
# import tensorflow_addons as tfa
from pathlib import Path
pd.options.display.max_columns=100

In [3]:
# VERSION_OUTPUT_PATH = Config.OUTPUT_PATH + os.sep + Config.VER
# Path(VERSION_OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

In [4]:
DATA_DIR = '/mnt/disks/extra_data/kaggle/ventilator_prediction/'
# submission = pd.read_csv(Config.DATA_PATH + os.sep + 'sample_submission.csv')
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))

In [5]:
fltr_out = train['u_out'] == 0
target = train['pressure']

In [6]:
oof_df = train[['id']].copy()

In [7]:
np.random.seed(10)

if Config.SAMPLE_TRAIN_BREATH_IDS is not None:
    sampled = np.random.choice(train['breath_id'].unique(), size=Config.SAMPLE_TRAIN_BREATH_IDS, replace=False)
    print(train.shape)
    train = train[train['breath_id'].isin(sampled)].reset_index(drop=True)
    print(train.shape)

if Config.SAMPLE_TEST_BREATH_IDS is not None:
    sampled = np.random.choice(test['breath_id'].unique(), size=Config.SAMPLE_TEST_BREATH_IDS, replace=False)
    test = test[test['breath_id'].isin(sampled)].reset_index(drop=True)

In [8]:
train['bidc'] = train.groupby('breath_id').cumcount() + 1
test['bidc'] = test.groupby('breath_id').cumcount() + 1

In [9]:
for df_tmp in [train, test]:
    df_tmp['R_cat'] = df_tmp['R'].map({20: 1, 50: 2, 5: 0})
    df_tmp['C_cat'] = df_tmp['C'].map({20: 1, 50: 2, 10: 0})
    df_tmp['RC_cat'] = df_tmp['R'].astype('str')+'-' + df_tmp['C'].astype('str')

mapper = pd.Series(index=train['RC_cat'].unique(), data=np.arange(train['RC_cat'].nunique()))
for df_tmp in [train, test]:
    df_tmp['RC_cat'] = df_tmp['RC_cat'].map(mapper)


cat_cols = ['R_cat', 'C_cat', 'RC_cat']
cat_cols_unq_dct = {}
for c in cat_cols:
    cat_cols_unq_dct[c] = train[c].nunique()

In [10]:
# c = 'u_in'
# LAG_WINDOW_RANGE = range(3)
# train = train.assign(**{f'{c}_t-{t}': train.groupby('breath_id')[c].shift(t) for t in LAG_WINDOW_RANGE})
# test = test.assign(**{f'{c}_t-{t}': test.groupby('breath_id')[c].shift(t) for t in LAG_WINDOW_RANGE})
# use_fts = [f'{c}_t-{t}' for t in LAG_WINDOW_RANGE]

In [11]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,bidc,R_cat,C_cat,RC_cat
0,1,1,20,50,0.000000,0.083334,0,5.837492,1,1,2,0
1,2,1,20,50,0.033652,18.383041,0,5.907794,2,1,2,0
2,3,1,20,50,0.067514,22.509278,0,7.876254,3,1,2,0
3,4,1,20,50,0.101542,22.808822,0,11.742872,4,1,2,0
4,5,1,20,50,0.135756,25.355850,0,12.234987,5,1,2,0


In [12]:
for df_tmp in [train]:
    df_tmp['bidc'] = df_tmp.groupby('breath_id').cumcount()
    df_tmp['u_in_lag_1'] = df_tmp.groupby('breath_id')['u_in'].shift(1).fillna(-999)
    df_tmp['u_in_lag_2'] = df_tmp.groupby('breath_id')['u_in'].shift(2).fillna(-999)
    df_tmp['u_in_lag_3'] = df_tmp.groupby('breath_id')['u_in'].shift(3).fillna(-999)
    df_tmp['u_in_lag_4'] = df_tmp.groupby('breath_id')['u_in'].shift(4).fillna(-999)

    df_tmp['u_in_cumsum'] = df_tmp.groupby('breath_id')['u_in'].cumsum()
    df_tmp['u_in_cumsum_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(1).fillna(-999)
    df_tmp['u_in_cumsum_lag_2'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(2).fillna(-999)
    df_tmp['u_in_cumsum_lag_1-u_in_cumsum_lag_2'] = df_tmp['u_in_cumsum_lag_1'] - df_tmp['u_in_cumsum_lag_2']

    df_tmp['u_in_cumsum_lag_3'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(3).fillna(0)
    df_tmp['u_in_cumsum_lag_4'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(4).fillna(0)

    df_tmp['u_in_cummean'] = df_tmp['u_in_cumsum']/(df_tmp.groupby('breath_id')['u_in'].cumcount() + 1)
    df_tmp['u_in_cummax'] = df_tmp.groupby('breath_id')['u_in'].cummax()
    df_tmp['next_u_in'] = df_tmp.groupby('breath_id')['u_in'].shift(-1).fillna(0)

    df_tmp['area'] = df_tmp['time_step'] * df_tmp['u_in']
    df_tmp['area'] = df_tmp.groupby('breath_id')['area'].cumsum()
    df_tmp['area_lag_1'] = df_tmp.groupby('breath_id')['area'].shift(1).fillna(0)
    df_tmp['area_lag_2'] = df_tmp.groupby('breath_id')['area'].shift(2).fillna(0)
    df_tmp['area_lead_1'] = df_tmp.groupby('breath_id')['area'].shift(-1).fillna(0)
    df_tmp['area_lead_2'] = df_tmp.groupby('breath_id')['area'].shift(-2).fillna(0)
    df_tmp['area_diff_lag_1'] = df_tmp['area'] - df_tmp['area_lag_1']
    df_tmp['area_diff_lead_1'] = df_tmp['area'] - df_tmp['area_lead_1']


    df_tmp['u_in_cumsum*time_step'] = df_tmp['u_in_cumsum']*df_tmp['time_step']
    df_tmp['u_in_cumsum*time_step_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*time_step'].shift(1).fillna(0)
    df_tmp['u_in_cumsum*time_step/c'] = df_tmp['u_in_cumsum*time_step']/df_tmp['C']
    df_tmp['u_in_cumsum*time_step/c_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*time_step/c'].shift(1).fillna(0)
    df_tmp['area/c'] = df_tmp['area']/df_tmp['C']
    df_tmp['area/c_lag_1'] = df_tmp.groupby('breath_id')['area/c'].shift(1).fillna(0)
    
    df_tmp['u_out_lag_1'] = df_tmp.groupby('breath_id')['u_out'].shift(1).fillna(0)

    df_tmp['time_step*u_out']= df_tmp['time_step']*df_tmp['u_out']

    df_tmp['R+C'] = df_tmp['R'] + df_tmp['C']
    df_tmp['R/C'] = df_tmp['R'] / df_tmp['C']
    df_tmp['u_in/C'] = df_tmp['u_in'] / df_tmp['C']
    df_tmp['u_in/R'] = df_tmp['u_in'] / df_tmp['R']
    df_tmp['u_in_cumsum/C'] = df_tmp['u_in_cumsum'] / df_tmp['C']
    df_tmp['u_in_cumsum/R'] = df_tmp['u_in_cumsum'] / df_tmp['R']
    df_tmp['area*R/C'] = df_tmp['area'] * df_tmp['R/C']
    df_tmp['u_in_cumsum*R/C'] = df_tmp['u_in_cumsum'] * df_tmp['R/C']
    df_tmp['u_in_cumsum*R/C_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*R/C'].shift(1).fillna(0)


    df_tmp['timestep_diff'] = (df_tmp['time_step'] - df_tmp.groupby('breath_id')['time_step'].shift(1)).fillna(0)
    df_tmp['u_in_diff'] = (df_tmp['u_in'] - df_tmp.groupby('breath_id')['u_in'].shift(1)).fillna(0)
    df_tmp['u_in_pct_change'] = (df_tmp['u_in_diff']/(df_tmp['u_in_lag_1'] + 1e-4)).fillna(0)
    df_tmp['u_in_diff_next'] = (df_tmp['u_in'] - df_tmp.groupby('breath_id')['u_in'].shift(-1)).fillna(0)
    df_tmp['u_in_log'] = np.log1p(df_tmp['u_in'])
    df_tmp['u_in_cumsum_log'] = np.log1p(df_tmp['u_in_cumsum'])
    df_tmp['u_in_lag_1_is_zero'] = (df_tmp['u_in_lag_1'] == 0)
    df_tmp['u_in_zero'] = (df_tmp['u_in_lag_1'] == 0)
    df_tmp['u_in_lead_1'] = df_tmp.groupby('breath_id')['u_in'].shift(-1)
    df_tmp['maop'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp[df_tmp['u_in_zero']].groupby('breath_id')['bidc'].min())
    df_tmp['spike'] = (df_tmp['u_in'] > df_tmp['u_in_lag_1']) & (df_tmp['u_in'] > df_tmp['u_in_lead_1'])
    df_tmp['u_in_lag_1_is_zero_cumsum'] = df_tmp.groupby('breath_id')['u_in_lag_1_is_zero'].cumsum()
    df_tmp['is_max_u_in'] = df_tmp['u_in'] == df_tmp.groupby('breath_id')['u_in'].transform('max')
    df_tmp['nki'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp.groupby('breath_id')['u_in'].apply(np.argmax))
    df_tmp['nki2'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp.groupby('breath_id')['u_in_cumsum'].apply(np.argmax))
    df_tmp['nki3'] = df_tmp['nki'] - df_tmp['nki2']
    df_tmp['nki4'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp[df_tmp['u_in_lag_1_is_zero']].groupby('breath_id')['bidc'].max())
    df_tmp['u_in_cummax - u_in'] = df_tmp['u_in_cummax'] - df_tmp['u_in']
    

In [13]:
train.shape

(6036000, 67)

In [14]:


train = train[train['bidc'] <= Config.SEQUENCE_LENGTH].reset_index(drop=True)
test = test[test['bidc'] <= Config.SEQUENCE_LENGTH].reset_index(drop=True)

sample_oof_df = train[['id']].copy()
sample_preds_df = test[['id']].copy()

In [15]:
train.shape

(3018000, 67)

In [16]:
_ = gc.collect()

In [17]:
num_cols = [c for c in train.columns if c not in ['id', 'pressure', 'breath_id', 'u_out'] + cat_cols]

train[num_cols] = train[num_cols].fillna(0)
# test[num_cols] = test[num_cols].fillna(0)

for c in tqdm(num_cols):
    RS = StandardScaler()
    train[c] = RS.fit_transform(train[[c]])[:, 0]
#     test[c] = RS.transform(test[[c]])[:, 0]

100%|██████████| 60/60 [00:14<00:00,  4.11it/s]


In [18]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,bidc,R_cat,C_cat,RC_cat,u_in_lag_1,u_in_lag_2,u_in_lag_3,u_in_lag_4,u_in_cumsum,u_in_cumsum_lag_1,u_in_cumsum_lag_2,u_in_cumsum_lag_1-u_in_cumsum_lag_2,u_in_cumsum_lag_3,u_in_cumsum_lag_4,u_in_cummean,u_in_cummax,next_u_in,area,area_lag_1,area_lag_2,area_lead_1,area_lead_2,area_diff_lag_1,area_diff_lead_1,u_in_cumsum*time_step,u_in_cumsum*time_step_lag_1,u_in_cumsum*time_step/c,u_in_cumsum*time_step/c_lag_1,area/c,area/c_lag_1,u_out_lag_1,time_step*u_out,R+C,R/C,u_in/C,u_in/R,u_in_cumsum/C,u_in_cumsum/R,area*R/C,u_in_cumsum*R/C,u_in_cumsum*R/C_lag_1,timestep_diff,u_in_diff,u_in_pct_change,u_in_diff_next,u_in_log,u_in_cumsum_log,u_in_lag_1_is_zero,u_in_zero,u_in_lead_1,maop,spike,u_in_lag_1_is_zero_cumsum,is_max_u_in,nki,nki2,nki3,nki4,u_in_cummax - u_in
0,1,1,-0.359072,1.394522,-1.685675,-0.614677,0,5.837492,-1.689278,1,2,0,-6.204765,-4.344579,-3.504065,-2.994913,-0.857938,-3.118732,-2.763682,-0.229643,-0.795328,-0.775243,-0.920852,-1.068380,0.424106,-0.683444,-0.665904,-0.648603,-0.697024,-0.704757,-0.566897,0.491921,-0.685961,-0.670334,-0.775451,-0.757956,-0.809349,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.543888,-0.422895,-0.964948,-0.522067,-0.604429,-0.733632,-0.720441,-6.080090,0.032266,-0.108135,-1.812876,-1.064578,-3.458379,-0.67504,-0.67504,0.424106,-1.923327,-0.380129,-0.660098,-0.16961,-0.695780,-1.683736,0.17441,-1.448229,-0.776598
1,2,1,-0.359072,1.394522,-1.597761,0.397359,0,5.907794,-1.602648,1,2,0,0.087395,-4.344579,-3.504065,-2.994913,-0.806998,-0.670142,-2.763682,6.115873,-0.795328,-0.775243,-0.412099,-0.494947,0.656989,-0.679151,-0.665904,-0.648603,-0.686688,-0.689295,-0.490275,0.380639,-0.684250,-0.670334,-0.774619,-0.757956,-0.806976,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.159510,-0.116007,-0.940831,-0.508587,-0.602902,-0.717240,-0.720365,0.261680,1.813803,-0.103604,-0.433589,0.995823,-1.442889,-0.67504,-0.67504,0.656989,-1.866924,-0.380129,-0.660098,-0.16961,-0.652397,-1.597482,0.17441,-1.373675,-0.776598
2,3,1,-0.359072,1.394522,-1.509299,0.625553,0,7.876254,-1.516018,1,2,0,0.202645,0.175893,-3.504065,-2.994913,-0.744623,-0.625088,-0.530394,-0.112886,-0.795328,-0.775243,-0.166038,-0.365648,0.673895,-0.668607,-0.661518,-0.648603,-0.670936,-0.666313,-0.378672,0.282288,-0.678342,-0.668559,-0.771747,-0.757093,-0.801147,-0.790600,-0.524734,-0.560718,0.681566,-0.732796,-0.072840,-0.046809,-0.911302,-0.492083,-0.599152,-0.697168,-0.703649,0.301218,0.433968,-0.108130,-0.061195,1.133688,-0.906754,-0.67504,-0.67504,0.673895,-1.810520,-0.380129,-0.660098,-0.16961,-0.609014,-1.511229,0.17441,-1.299121,-0.776598
3,4,1,-0.359072,1.394522,-1.420404,0.642119,0,11.742872,-1.429389,1,2,0,0.228632,0.258692,0.239309,-2.994913,-0.681419,-0.569921,-0.489302,-0.086679,-0.795088,-0.775243,-0.038844,-0.356262,0.817648,-0.652538,-0.650744,-0.644113,-0.647525,-0.635429,-0.280038,0.143209,-0.668124,-0.662430,-0.766780,-0.754114,-0.792264,-0.784604,-0.524734,-0.560718,0.681566,-0.732796,-0.066548,-0.041786,-0.881379,-0.475358,-0.593436,-0.676829,-0.683180,0.332444,0.061427,-0.108135,-0.279908,1.142732,-0.603956,-0.67504,-0.67504,0.817648,-1.754116,-0.380129,-0.660098,-0.16961,-0.565631,-1.424975,0.17441,-1.224567,-0.776598
4,5,1,-0.359072,1.394522,-1.331024,0.782979,0,12.234987,-1.342759,1,2,0,0.230518,0.277362,0.307875,0.292417,-0.611156,-0.514021,-0.438986,-0.084776,-0.742144,-0.775000,0.065797,-0.276449,0.925109,-0.628654,-0.634324,-0.633084,-0.616063,-0.598535,-0.140558,-0.002983,-0.652634,-0.651829,-0.759250,-0.748961,-0.779062,-0.775466,-0.524734,-0.560718,0.681566,-0.732796,-0.013049,0.000928,-0.848115,-0.456766,-0.584940,-0.654218,-0.662439,0.367400,0.280227,-0.108133,-0.217334,1.215333,-0.373495,-0.67504,-0.67504,0.925109,-1.697712,-0.380129,-0.660098,-0.16961,-0.522247,-1.338721,0.17441,-1.150013,-0.776598


In [18]:
temp = pd.read_pickle('../data/nm_processed_train.pkl')

In [19]:
temp.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,R_cat,C_cat,RC_cat,bidc,u_in_lag_1,u_in_lag_2,u_in_lag_3,u_in_lag_4,u_in_cumsum,u_in_cumsum_lag_1,u_in_cumsum_lag_2,u_in_cumsum_lag_1-u_in_cumsum_lag_2,u_in_cumsum_lag_3,u_in_cumsum_lag_4,u_in_cummean,u_in_cummax,next_u_in,area,area_lag_1,area_lag_2,area_lead_1,area_lead_2,area_diff_lag_1,area_diff_lead_1,u_in_cumsum*time_step,u_in_cumsum*time_step_lag_1,u_in_cumsum*time_step/c,u_in_cumsum*time_step/c_lag_1,area/c,area/c_lag_1,u_out_lag_1,time_step*u_out,R+C,R/C,u_in/C,u_in/R,u_in_cumsum/C,u_in_cumsum/R,area*R/C,u_in_cumsum*R/C,u_in_cumsum*R/C_lag_1,timestep_diff,u_in_diff,u_in_pct_change,u_in_diff_next,u_in_log,u_in_cumsum_log,u_in_lag_1_is_zero,u_in_zero,u_in_lead_1,maop,spike,u_in_lag_1_is_zero_cumsum,is_max_u_in,nki,nki2,nki3,nki4,u_in_cummax - u_in
0,1,1,-0.359072,1.394522,-1.685675,-0.614677,0,110,1,2,0,-1.689278,-6.204765,-4.344579,-3.504065,-2.994913,-0.857938,-3.118732,-2.763682,-0.229643,-0.795328,-0.775243,-0.920852,-1.068380,0.424106,-0.683444,-0.665904,-0.648603,-0.697024,-0.704757,-0.566897,0.491921,-0.685961,-0.670334,-0.775451,-0.757956,-0.809349,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.543888,-0.422895,-0.964948,-0.522067,-0.604429,-0.733632,-0.720441,-6.080090,0.032266,-0.108135,-1.812876,-1.064578,-3.458379,-0.67504,-0.67504,0.424106,-1.923327,-0.380129,-0.660098,-0.16961,-0.695780,-1.683736,0.17441,-1.448229,-0.776598
1,2,1,-0.359072,1.394522,-1.597761,0.397359,0,111,1,2,0,-1.602648,0.087395,-4.344579,-3.504065,-2.994913,-0.806998,-0.670142,-2.763682,6.115873,-0.795328,-0.775243,-0.412099,-0.494947,0.656989,-0.679151,-0.665904,-0.648603,-0.686688,-0.689295,-0.490275,0.380639,-0.684250,-0.670334,-0.774619,-0.757956,-0.806976,-0.793041,-0.524734,-0.560718,0.681566,-0.732796,-0.159510,-0.116007,-0.940831,-0.508587,-0.602902,-0.717240,-0.720365,0.261680,1.813803,-0.103604,-0.433589,0.995823,-1.442889,-0.67504,-0.67504,0.656989,-1.866924,-0.380129,-0.660098,-0.16961,-0.652397,-1.597482,0.17441,-1.373675,-0.776598
2,3,1,-0.359072,1.394522,-1.509299,0.625553,0,139,1,2,0,-1.516018,0.202645,0.175893,-3.504065,-2.994913,-0.744623,-0.625088,-0.530394,-0.112886,-0.795328,-0.775243,-0.166038,-0.365648,0.673895,-0.668607,-0.661518,-0.648603,-0.670936,-0.666313,-0.378672,0.282288,-0.678342,-0.668559,-0.771747,-0.757093,-0.801147,-0.790600,-0.524734,-0.560718,0.681566,-0.732796,-0.072840,-0.046809,-0.911302,-0.492083,-0.599152,-0.697168,-0.703649,0.301218,0.433968,-0.108130,-0.061195,1.133688,-0.906754,-0.67504,-0.67504,0.673895,-1.810520,-0.380129,-0.660098,-0.16961,-0.609014,-1.511229,0.17441,-1.299121,-0.776598
3,4,1,-0.359072,1.394522,-1.420404,0.642119,0,194,1,2,0,-1.429389,0.228632,0.258692,0.239309,-2.994913,-0.681419,-0.569921,-0.489302,-0.086679,-0.795088,-0.775243,-0.038844,-0.356262,0.817648,-0.652538,-0.650744,-0.644113,-0.647525,-0.635429,-0.280038,0.143209,-0.668124,-0.662430,-0.766780,-0.754114,-0.792264,-0.784604,-0.524734,-0.560718,0.681566,-0.732796,-0.066548,-0.041786,-0.881379,-0.475358,-0.593436,-0.676829,-0.683180,0.332444,0.061427,-0.108135,-0.279908,1.142732,-0.603956,-0.67504,-0.67504,0.817648,-1.754116,-0.380129,-0.660098,-0.16961,-0.565631,-1.424975,0.17441,-1.224567,-0.776598
4,5,1,-0.359072,1.394522,-1.331024,0.782979,0,201,1,2,0,-1.342759,0.230518,0.277362,0.307875,0.292417,-0.611156,-0.514021,-0.438986,-0.084776,-0.742144,-0.775000,0.065797,-0.276449,0.925109,-0.628654,-0.634324,-0.633084,-0.616063,-0.598535,-0.140558,-0.002983,-0.652634,-0.651829,-0.759250,-0.748961,-0.779062,-0.775466,-0.524734,-0.560718,0.681566,-0.732796,-0.013049,0.000928,-0.848115,-0.456766,-0.584940,-0.654218,-0.662439,0.367400,0.280227,-0.108133,-0.217334,1.215333,-0.373495,-0.67504,-0.67504,0.925109,-1.697712,-0.380129,-0.660098,-0.16961,-0.522247,-1.338721,0.17441,-1.150013,-0.776598


In [18]:
train.shape

(3093450, 67)

In [19]:
train.groupby('breath_id').size()

breath_id
1         41
2         41
3         41
4         41
5         41
          ..
125740    41
125742    41
125743    41
125745    41
125749    41
Length: 75450, dtype: int64

In [ ]:
_ = gc.collect()

In [ ]:
N_CLASSES = train['pressure'].nunique()

In [ ]:
class VentilatorDataset:
    def __init__(self, df, cat_cols, num_cols, is_train=True):

        if is_train:
            self.inv_mapper = df['pressure'].drop_duplicates().sort_values().reset_index(drop=True).to_dict()
            self.mapper = {val: key for key, val in self.inv_mapper.items()}
            df['pressure_int'] = df['pressure'].map(self.mapper)
            self.pressures_int = df[['pressure_int']].to_numpy().reshape(-1, df['bidc'].nunique())
            self.pressures = df[['pressure']].to_numpy().reshape(-1, df['bidc'].nunique())
            del self.mapper
            _ = gc.collect()

        self.u_outs = df[['u_out']].to_numpy().reshape(-1, df['bidc'].nunique())
        self.inputs = df[num_cols + cat_cols].values.reshape(-1, df['bidc'].nunique(), len(num_cols + cat_cols)).astype(np.float32)

In [ ]:
%%time
test_data = VentilatorDataset(test, cat_cols, num_cols, is_train=False)
del test
_ = gc.collect()
train_data = VentilatorDataset(train, cat_cols, num_cols, is_train=True)
del train
_ = gc.collect()

CPU times: user 2.6 s, sys: 1.61 s, total: 4.21 s
Wall time: 4.17 s


In [ ]:
train_data.inputs.shape, train_data.pressures.shape, train_data.u_outs.shape

((75450, 40, 66), (75450, 40), (75450, 40))

In [ ]:
test_data.inputs.shape, test_data.u_outs.shape

((50300, 40, 66), (50300, 40))

In [ ]:
NUMERICAL_INPUTS_CNT = len(num_cols)
N_FEATS = train_data.inputs.shape[2]
WINDOW_SIZE=train_data.inputs.shape[1]

NUMERICAL_INPUTS_CNT

63

In [ ]:
mapper = train_data.inv_mapper
def map_class_to_cont(x):
    return mapper[x]

In [ ]:
class WeightedSum(tf.keras.layers.Layer):
    """A custom keras layer to learn a weighted sum of tensors"""

    def __init__(self, **kwargs):
        super(WeightedSum, self).__init__(**kwargs)

    def build(self, input_shape=1):
        self.a = self.add_weight(
            name='alpha',
            shape=(),
            initializer='ones',
            dtype='float32',
            trainable=True)

    def call(self, model_outputs):
        return self.a * model_outputs[0] + model_outputs[1]

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = inputs
    x = tfa.layers.MultiHeadAttention(
        head_size=head_size,
        num_heads=num_heads,
        use_projection_bias = False,
        dropout=Config.DROPOUT
    )([x, x, x])

    res = WeightedSum()([x, inputs])
    res = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)

    # Feed Forward Part
    x = tf.keras.layers.Dense(ff_dim, activation='relu')(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)

    x = tf.keras.layers.Dense(inputs.shape[-1])(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)
    x = WeightedSum()([x, res])

    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

In [ ]:
def custom_mean_absolute_error(y_true, y_pred, w_out):
    return K.mean(K.abs(y_pred - y_true))

def custom_metric(y_true, y_pred, w_out):
    return mae(y_true[w_out == 0], y_pred[w_out == 0])

def get_LSTM_model(Config):
    w_out = tf.keras.layers.Input(shape=(WINDOW_SIZE, 1))
    targets = tf.keras.layers.Input(shape=(WINDOW_SIZE, 1))
    inputs = tf.keras.layers.Input(shape=(WINDOW_SIZE, N_FEATS))
    
    num_inputs = inputs[:, :, :NUMERICAL_INPUTS_CNT]
    cat_inputs = inputs[:, :, NUMERICAL_INPUTS_CNT:]

    initializer = tf.keras.initializers.glorot_uniform(seed=66)

    all_conv_layers = []
    for sz in [2, 3, 5, 7, 11, 15]:
        conv_curr = tf.keras.layers.Conv1D(16, kernel_size=(sz,), padding='same', use_bias=False)(num_inputs)
        # conv_curr = tf.keras.layers.LayerNormalization()(conv_curr)
        all_conv_layers.append(conv_curr)

    conv_op = tf.keras.layers.Concatenate()(all_conv_layers)
    num_inputs = tf.keras.layers.Concatenate()([num_inputs, conv_op])

    for i, c in enumerate(cat_cols):
      cat_inp = cat_inputs[:, :, i:i+1]
      embed = tf.keras.layers.Embedding(input_dim=cat_cols_unq_dct[c]+2, output_dim=Config.CAT_DIM, embeddings_initializer=initializer)(cat_inp)
      reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
      reshaped = tf.keras.layers.SpatialDropout1D(Config.EMBEDDING_DROPOUT, seed=2)(reshaped)
      num_inputs = tf.keras.layers.Concatenate(axis=2)([reshaped, num_inputs])
    
    
    x = num_inputs
    for i in range(16):
        x = transformer_encoder(x, 8, 128, 1024, 0)
        x_c = tf.keras.layers.Conv1D(16, kernel_size=(3,), padding='same', use_bias=False)(x)
        x = tf.keras.layers.Concatenate()([x, x_c])

         
    x = tf.keras.layers.Dense(Config.DENSE_SIZE)(x)
    x = tfa.activations.mish(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)
    out = tf.keras.layers.Dense(N_CLASSES)(x)
    
    model = tf.keras.Model(inputs=[inputs, w_out], outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    
    return model

In [ ]:

def get_preds(best_model, X_test, wt_test, STEP=5000):
    test_preds = []
    for start in range(0, X_test.shape[0], STEP):
        preds = best_model.predict([X_test[start: start+STEP], wt_test[start: start+STEP]], verbose=0,
                                   batch_size=STEP, use_multiprocessing=False)
        preds = np.argmax(preds, axis=2)
        test_preds.append(preds)

    test_preds = np.concatenate(test_preds, axis=0)
    return test_preds

In [ ]:

class IntervalEvaluation(tf.keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=10, save_model=True, Config={}, fold=0):

        super(tf.keras.callbacks.Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val, self.w_out_val = validation_data
        self.best_score = np.inf
        self.save_model = save_model
        self.learning_rates = pd.Series(index=np.arange(Config.N_EPOCHS),
                                        data=np.linspace(Config.END_LR,
                                                         Config.START_LR,
                                                         num=Config.N_EPOCHS)[::-1]).to_dict()

    def on_epoch_begin(self, epoch, logs={}):
        self.start_time = time()

        if epoch == 0:
            self.first_epoch_start_time = self.start_time

        if epoch in self.learning_rates:
            tf.keras.backend.set_value(self.model.optimizer.lr, self.learning_rates[epoch])

        self.curr_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
    def on_epoch_end(self, epoch, logs={}):
        keys = list(logs.keys())

        y_pred = get_preds(self.model, self.X_val, self.w_out_val, STEP=3000)
        y_pred = np.vectorize(map_class_to_cont)(y_pred)
        _ = gc.collect()
        
        val_score = custom_metric(self.y_val.flatten(), y_pred.flatten(), self.w_out_val.flatten())
        
        if val_score < self.best_score:
            self.best_score = val_score
            if self.save_model and (epoch == 0 or epoch > 150): ##### Saving models takes time, so don't save for the first N_EPOCHS // 2
                tf.keras.models.save_model(self.model, f'best_model', save_format="h5")

        total_time = round(time() - self.start_time, 2)
        total_seconds_till_now = round(time() - self.first_epoch_start_time, 0)

        if Config.WANDB_ENABLED:
            wandb.log({f'Fold {fold} epoch': epoch, f"Fold {fold} train_loss": logs['loss'], f"Fold {fold} val_score": val_score, f'Fold {fold} best_val_score': self.best_score, f'Fold {fold} total_time_in_seconds': total_time})
        
        if epoch % self.interval == 0:
            print(f"Epoch: {epoch:03d} curr_lr: {self.curr_lr:.1e} - train_loss: {logs['loss']:.03f} val_score: {val_score:.03f}  best_val_score: {self.best_score:.03f}  last_epoch t={total_time:.02f}s, total_time_elapsed t={total_seconds_till_now}s")

In [ ]:
Config_dct = {key:value for key, value in Config.__dict__.items() if not key.startswith('__') and not callable(key)}
if Config.WANDB_ENABLED:
    import wandb
    wandb.login(key=Config.WANDB_API_KEY)
    wandb.init(project="google-brain",
               name=f'nikhil_pressure_prediction_dl_{Config.VER}',
               save_code=True,
               allow_val_change=True,
               config=Config_dct)

In [ ]:
folds_lst = list(range(Config.START_FOLD, Config.END_FOLD+1))
folds_lst

[0, 1]

In [ ]:
def save_test_preds(preds, path):

  global submission
 
  sample_preds_df['pressure'] = preds.flatten()
  submission = pd.merge(submission[['id']], sample_preds_df, on='id', how='left')
  submission['pressure'] = submission['pressure'].fillna(0)
  
  submission.to_csv(path, index=False)

In [32]:
oofs = np.zeros(train_data.pressures.shape)
kf = KFold(n_splits=Config.N_SPLITS, shuffle=True, random_state=Config.RANDOM_STATE)
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

preds_df_tmp = pd.DataFrame()
oofs_df_tmp = pd.DataFrame()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    for fold, (trn_idx, val_idx) in enumerate(kf.split(train_data.pressures, train_data.pressures)):


      if fold not in folds_lst:
        continue

      print('-'*15, '>', f'Fold {fold}', '<', '-'*15, '\n')

      X_trn, X_val = train_data.inputs[trn_idx], train_data.inputs[val_idx]
      y_trn, y_val = train_data.pressures[trn_idx], train_data.pressures[val_idx]
      y_trn_int, y_val_int = train_data.pressures_int[trn_idx], train_data.pressures_int[val_idx]

      wt_trn, wt_val = train_data.u_outs[trn_idx], train_data.u_outs[val_idx]

      model = get_LSTM_model(Config)

      print_val_results = IntervalEvaluation(
          validation_data=(X_val, y_val, wt_val),
          interval=1,
          Config=Config,
          fold=fold,
          )

      history = model.fit([X_trn, wt_trn], y_trn_int,
                  verbose=0,
                  epochs=Config.N_EPOCHS,
                  batch_size=Config.BATCH_SIZE,
                  callbacks=[print_val_results]
                  )

      del model, X_trn, y_trn, wt_trn, print_val_results
      _ = gc.collect()

      custom_objects = {
          'custom_mean_absolute_error': custom_mean_absolute_error,
          'WeightedSum': WeightedSum,
      }

      best_model = load_model('best_model', custom_objects=custom_objects)

      vp = get_preds(best_model, X_val, wt_val, STEP=3000)
      vp = np.vectorize(map_class_to_cont)(vp)
      np.save(VERSION_OUTPUT_PATH + os.sep + f'val_fold_{fold}.npy', vp)

      _ = gc.collect()

      val_score = round(custom_metric(y_val.flatten(), vp.flatten(), wt_val.flatten()), 4)
      print(f'\nMAE val: {val_score}')

      tp = get_preds(best_model, test_data.inputs, test_data.u_outs, STEP=3000)
      tp = np.vectorize(map_class_to_cont)(tp)

      path = VERSION_OUTPUT_PATH + os.sep + f'sub_fold_{fold}.csv'
      save_test_preds(tp, path)

      del X_val, y_val, wt_val, best_model, vp, tp, trn_idx, val_idx
      _ = gc.collect()

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.105.186.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.105.186.218:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


--------------- > Fold 0 < --------------- 

Epoch: 000 curr_lr: 2.0e-04 - train_loss: 5.002 val_score: 1.398  best_val_score: 1.398  last_epoch t=313.82s, total_time_elapsed t=314.0s
Epoch: 001 curr_lr: 2.0e-04 - train_loss: 4.044 val_score: 0.904  best_val_score: 0.904  last_epoch t=74.93s, total_time_elapsed t=389.0s
Epoch: 002 curr_lr: 2.0e-04 - train_loss: 3.762 val_score: 0.709  best_val_score: 0.709  last_epoch t=75.35s, total_time_elapsed t=464.0s
Epoch: 003 curr_lr: 2.0e-04 - train_loss: 3.609 val_score: 0.644  best_val_score: 0.644  last_epoch t=74.93s, total_time_elapsed t=539.0s
Epoch: 004 curr_lr: 2.0e-04 - train_loss: 3.496 val_score: 0.577  best_val_score: 0.577  last_epoch t=75.18s, total_time_elapsed t=614.0s
Epoch: 005 curr_lr: 2.0e-04 - train_loss: 3.415 val_score: 0.516  best_val_score: 0.516  last_epoch t=75.88s, total_time_elapsed t=690.0s
Epoch: 006 curr_lr: 2.0e-04 - train_loss: 3.345 val_score: 0.516  best_val_score: 0.516  last_epoch t=75.22s, total_time_elaps